# PREDICTING ASTEROID DIAMETER

>by Dr Juan H Klopper

- Research Fellow
- School for Data Science and Computational Thinking
- Stellenbosch University

## PACKAGES USED IN THIS NOTEBOOK

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
import plotly.graph_objects as go
import plotly.io as pio

pio.templates.default = 'plotly_white'

In [ ]:
!pip install tensorflow_decision_forests --upgrade -q

## DATA IMPORT

| VARIABLE NAME  | DESCRIPTION                                      |
| :--------------| :------------------------------------------------|
| a              | semi-major axis[au]                              |
| e              | eccentricity                                     |
| i              | inclination wrt x-y ecliptic plane [deg]         |
| om             | longitude of the ascending node                  |
| w              | argument of perihelion                           |
| q              | perihelion distance [au]                         |
| ad             | aphelion distance [au]                           |
| per_y          | orbital period [years]                           |
| data_arc       | data arc-span [d]                                |
| condition_code | orbit condition code                             |
| n_obs_use      | number of observations used                      |
| H              | absolute magnitude parameter                     |
| diameter       | diameter of asteroid [km]                        |
| extent         | object bi or tri-axial ellipsoid dimensions [km] |
| albedo         | geometric albedo                                 |
| rot_per        | standard gravitational parameter [$m \times G$]  |
| bv             | color index B-V magnitude difference             |
| ub             | color index U-B magnitude difference             |
| IR             | color index I-R magnitude difference             |
| spec_B         | spectral taxonomic type (SMASSII)                |
| spec_T         | spectral taxonomic type (Tholen)                 |
| neo            | near earth object                                |
| pha            | physically hazardous asteroid                    |
| moid           | earth minimum orbit intersection distance [au]   |

In [ ]:
df = pd.read_csv('/kaggle/input/prediction-of-asteroid-diameter/Asteroid.csv', low_memory=False)

In [ ]:
df.info()

In [ ]:
df = df.dropna(subset=['diameter'])

In [ ]:
df.info()

NASA uses the absolute magnitude parameter `H` and geometric albedo `albedo` in their model.

In [ ]:
df = df.drop(columns=['full_name', 'H', 'albedo', 'G', 'extent', 'rot_per', 'GM', 'BV', 'UB', 'IR', 'spec_B', 'spec_T'])
df.info()

In [ ]:
df.diameter.dtype

In [ ]:
df.diameter = pd.to_numeric(df.diameter, errors='coerce')

In [ ]:
np.sum(df.diameter.isna())

In [ ]:
df = df.dropna(subset=['diameter'])
df.info()

In [ ]:
df.diameter.describe()

In [ ]:
df = df.dropna(subset=['data_arc'])
df.info()

In [ ]:
df.condition_code.value_counts()

In [ ]:
df.condition_code = pd.to_numeric(df.condition_code, errors='coerce')

In [ ]:
df.neo.value_counts()

In [ ]:
df.neo = df.neo.replace({'N':0, 'Y':1})

In [ ]:
df.pha.value_counts()

In [ ]:
df.pha = df.pha.replace({'N':0, 'Y':1})

In [ ]:
df.info()

In [ ]:
for column in df.columns:
    if df[column].dtype == float:
        df[column] = df[column].astype('float32')

## TRAIN TEST SPLIT

In [ ]:
features = df.drop('diameter', axis=1)
target = df.diameter

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features,
                                                   target,
                                                   test_size=0.2,
                                                   random_state=28)

## TRAIN

In [ ]:
forest = RandomForestRegressor(max_depth=32,
                              n_estimators=50)

In [ ]:
X_train

In [ ]:
for column in df.columns:
    print(column, np.sum(df[column].isna()))

In [ ]:
forest.fit(X_train,
          np.ravel(y_train))

## PREDICTION

In [ ]:
y_pred = forest.predict(X_test)

## METRICS

In [ ]:
r2_score(y_test,
        y_pred)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=y_test,
                        y=y_pred,
                        mode='markers'))

fig.update_layout(title='Actual vs predicted diameters',
                 xaxis=dict(title='Actual diameters'),
                 yaxis=dict(title='Predicted diameters'))